In [2]:
import duckdb

# DuckDB veritabanını başlat
conn = duckdb.connect()

# Veriyi csv dosyasından yükle
conn.execute("CREATE TABLE capstone_data AS SELECT * FROM read_csv('capstone_data_filled_new.csv')")

# İlk 5 satırı göster
result = conn.execute("SELECT * FROM capstone_data LIMIT 5").fetchall()

# Sonuçları yazdır
for row in result:
    print(row)

('51893f29-e6c2-45d3-807c-e1280d3d7b90', 18, 53.0, 'Prepaid', 106.74, 139.72, 47.31, 59.45, 0, 0, 25, 18.0, 13, 1.34, '[]', 0, 1, 0, 0, 0, 0, 0, 0, 0)
('a568caf1-d851-4847-a9f5-20ef9017fa92', 26, 15.0, 'Prepaid', 31.55, 12.14, 21.52, 1221.65, 0, 0, 51, 8.0, 3, 2.5700000000000003, '[]', 0, 1, 0, 0, 0, 0, 0, 0, 0)
('c611bf0e-a013-44dc-9939-bd33dab16d14', 32, 152.0, 'Postpaid', 30.64, 10.17, 31.11, 1170.45, 0, 1, 0, 13.0, 10, 8.02, '[]', 0, 0, 0, 1, 0, 0, 0, 0, 0)
('9284b7b8-a4ef-49a7-9fa4-333954491f57', 33, 35.0, 'Prepaid', 60.41131400092014, 99.8417508662536, 36.03, 2418.91, 0, 0, 19, 7.0, 11, 5.96, "['RitimGo']", 0, 1, 0, 0, 0, 0, 0, 0, 0)
('e205b674-a6e6-43c5-aed7-9497c37c5c82', 18, 243.0, 'Prepaid', 85.62, 164.79, 46.4, 1518.19, 0, 0, 99, 15.0, 6, 8.29, '[]', 0, 1, 0, 0, 0, 0, 0, 0, 0)


In [3]:
from prettytable import PrettyTable

# Sütun isimlerini al
columns = [col[1] for col in conn.execute("PRAGMA table_info(capstone_data)").fetchall()]

# Veriyi al
result = conn.execute("SELECT * FROM capstone_data LIMIT 5").fetchall()

# PrettyTable ile tabloyu oluştur
table = PrettyTable()
table.field_names = columns

# Sonuçları tablonun satırları olarak ekle
for row in result:
    table.add_row(row)

# Tabloyu yazdır
print(table)

+--------------------------------------+-----+--------+--------------+-------------------+------------------+---------------+----------------+------------------+--------------+------------------+------------+------------------------+--------------------+-------------+-------+----------------------+------------------------+-----------------------+---------------+----------------------+--------------------+-----------------------+-------------------+
|                  id                  | age | tenure | service_type | avg_call_duration |    data_usage    | roaming_usage | monthly_charge | overdue_payments | auto_payment | avg_top_up_count | call_drops | customer_support_calls | satisfaction_score |     apps    | churn | service_type_Prepaid | service_type_Broadband | service_type_Postpaid | apps_tuple_() | apps_tuple_Konusalim | apps_tuple_CuzdanX | apps_tuple_HizliPazar | apps_tuple_IzleGo |
+--------------------------------------+-----+--------+--------------+-------------------+----

In [4]:
# Yoğun Kullanıcı (Heavy User) Belirleme
conn.execute("""
ALTER TABLE capstone_data ADD COLUMN heavy_user BOOLEAN;
UPDATE capstone_data 
SET heavy_user = CASE 
    WHEN avg_call_duration > (SELECT AVG(avg_call_duration) FROM capstone_data) 
      OR data_usage > (SELECT AVG(data_usage) FROM capstone_data) 
      OR roaming_usage > (SELECT AVG(roaming_usage) FROM capstone_data) 
    THEN TRUE ELSE FALSE 
END;
""")

In [5]:
# Ödeme Disiplini (Bad Payer) Belirleme

conn.execute("""
ALTER TABLE capstone_data ADD COLUMN bad_payer BOOLEAN;
UPDATE capstone_data 
SET bad_payer = CASE 
    WHEN overdue_payments > 2 AND auto_payment = 0 THEN TRUE
    ELSE FALSE 
END;
""")

In [6]:
# Müşteri Memnuniyetsizliği (Unhappy Customer) Belirleme

conn.execute("""
ALTER TABLE capstone_data ADD COLUMN unhappy_customer BOOLEAN;
UPDATE capstone_data 
SET unhappy_customer = CASE 
    WHEN customer_support_calls > 5 AND satisfaction_score < 3 THEN TRUE
    ELSE FALSE 
END;
""")

In [7]:
# Kaç Farklı Uygulama Kullandığını Belirleme

conn.execute("""
ALTER TABLE capstone_data ADD COLUMN num_of_apps_used INTEGER;
UPDATE capstone_data 
SET num_of_apps_used = apps_tuple_Konusalim + apps_tuple_CuzdanX + 
                       apps_tuple_HizliPazar + apps_tuple_IzleGo;
""")

In [13]:
df = conn.execute("""
SELECT * FROM capstone_data
""").fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [14]:
df.head()

,id,age,tenure,service_type,avg_call_duration,data_usage,roaming_usage,monthly_charge,overdue_payments,auto_payment,...,apps_tuple_(),apps_tuple_Konusalim,apps_tuple_CuzdanX,apps_tuple_HizliPazar,apps_tuple_IzleGo,heavy_user,bad_payer,unhappy_customer,num_of_apps_used,tenure_category
0,51893f29-e6c2-45d3-807c-e1280d3d7b90,18,53.0,Prepaid,106.740000,139.720000,47.31,59.45,0,0,...,0,0,0,0,0,True,False,True,0,Loyal
1,a568caf1-d851-4847-a9f5-20ef9017fa92,26,15.0,Prepaid,31.550000,12.140000,21.52,1221.65,0,0,...,0,0,0,0,0,False,False,False,0,Loyal
2,c611bf0e-a013-44dc-9939-bd33dab16d14,32,152.0,Postpaid,30.640000,10.170000,31.11,1170.45,0,1,...,0,0,0,0,0,True,False,False,0,Very Loyal
3,9284b7b8-a4ef-49a7-9fa4-333954491f57,33,35.0,Prepaid,60.411314,99.841751,36.03,2418.91,0,0,...,0,0,0,0,0,True,False,False,0,Loyal
4,e205b674-a6e6-43c5-aed7-9497c37c5c82,18,243.0,Prepaid,85.620000,164.790000,46.40,1518.19,0,0,...,0,0,0,0,0,True,False,False,0,Very Loyal


In [15]:
import pandas as pd

# Boolean sütunları (True/False) için one-hot encoding
df['heavy_user'] = df['heavy_user'].astype(int)
df['bad_payer'] = df['bad_payer'].astype(int)
df['unhappy_customer'] = df['unhappy_customer'].astype(int)

# 'tenure_category' için one-hot encoding
df = pd.get_dummies(df, columns=['tenure_category'], drop_first=False)

In [18]:
df.head()

,id,age,tenure,service_type,avg_call_duration,data_usage,roaming_usage,monthly_charge,overdue_payments,auto_payment,...,apps_tuple_CuzdanX,apps_tuple_HizliPazar,apps_tuple_IzleGo,heavy_user,bad_payer,unhappy_customer,num_of_apps_used,tenure_category_Loyal,tenure_category_New,tenure_category_Very Loyal
0,51893f29-e6c2-45d3-807c-e1280d3d7b90,18,53.0,Prepaid,106.740000,139.720000,47.31,59.45,0,0,...,0,0,0,1,0,1,0,True,False,False
1,a568caf1-d851-4847-a9f5-20ef9017fa92,26,15.0,Prepaid,31.550000,12.140000,21.52,1221.65,0,0,...,0,0,0,0,0,0,0,True,False,False
2,c611bf0e-a013-44dc-9939-bd33dab16d14,32,152.0,Postpaid,30.640000,10.170000,31.11,1170.45,0,1,...,0,0,0,1,0,0,0,False,False,True
3,9284b7b8-a4ef-49a7-9fa4-333954491f57,33,35.0,Prepaid,60.411314,99.841751,36.03,2418.91,0,0,...,0,0,0,1,0,0,0,True,False,False
4,e205b674-a6e6-43c5-aed7-9497c37c5c82,18,243.0,Prepaid,85.620000,164.790000,46.40,1518.19,0,0,...,0,0,0,1,0,0,0,False,False,True


In [21]:
df['tenure_category_Loyal'] = df['tenure_category_Loyal'].astype(int)
df['tenure_category_Very Loyal'] = df['tenure_category_Very Loyal'].astype(int)
df['tenure_category_New'] = df['tenure_category_New'].astype(int)

In [22]:
df.head()

,id,age,tenure,service_type,avg_call_duration,data_usage,roaming_usage,monthly_charge,overdue_payments,auto_payment,...,apps_tuple_CuzdanX,apps_tuple_HizliPazar,apps_tuple_IzleGo,heavy_user,bad_payer,unhappy_customer,num_of_apps_used,tenure_category_Loyal,tenure_category_New,tenure_category_Very Loyal
0,51893f29-e6c2-45d3-807c-e1280d3d7b90,18,53.0,Prepaid,106.740000,139.720000,47.31,59.45,0,0,...,0,0,0,1,0,1,0,1,0,0
1,a568caf1-d851-4847-a9f5-20ef9017fa92,26,15.0,Prepaid,31.550000,12.140000,21.52,1221.65,0,0,...,0,0,0,0,0,0,0,1,0,0
2,c611bf0e-a013-44dc-9939-bd33dab16d14,32,152.0,Postpaid,30.640000,10.170000,31.11,1170.45,0,1,...,0,0,0,1,0,0,0,0,0,1
3,9284b7b8-a4ef-49a7-9fa4-333954491f57,33,35.0,Prepaid,60.411314,99.841751,36.03,2418.91,0,0,...,0,0,0,1,0,0,0,1,0,0
4,e205b674-a6e6-43c5-aed7-9497c37c5c82,18,243.0,Prepaid,85.620000,164.790000,46.40,1518.19,0,0,...,0,0,0,1,0,0,0,0,0,1


In [23]:
df.to_csv('encoded_data.csv', index=False)

In [24]:
conn.close()